# Principles of Machine Learning - Final Exam - Classifier

The task of the classifier is predict the likeliness that a potential new customer will buy a bike from our company as well.

Our data foundation is a set of 16,400 previous customers.

In [118]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import math
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [120]:
restored = pd.read_csv("ExportedFeatures.csv")
restored_buyers = pd.read_csv("ExportedBuyers.csv")
restored_spend = pd.read_csv("ExportedSpend.csv")
restored_test = pd.read_csv("ExportedTest.csv")
Features = restored.to_numpy()
buyers = restored_buyers.to_numpy()
spend = restored_spend.to_numpy()
Test_Features = restored_test.to_numpy()

# Training

In [122]:
labels = np.array(buyers)
# ohe = preprocessing.OneHotEncoder(categories='auto')
# encoded = ohe.fit(labels.reshape(-1,1))
# labels = encoded.transform(labels.reshape(-1,1))

X_train, X_test, y_train, y_test = ms.train_test_split(Features, labels, test_size=0.33, random_state=42)

print("{} {} {} {}".format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

logistic_mod = linear_model.LogisticRegression(max_iter=100) 
logistic_mod.fit(X_train, y_train)

def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])


def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

probabilities = logistic_mod.predict_proba(X_test)
print(probabilities[:15,:])
scores = score_model(probabilities, 0.5)    
print_metrics(y_test, scores)

(10990, 26) (5414, 26) (10990, 1) (5414, 1)
[[0.82757764 0.17242236]
 [0.98277459 0.01722541]
 [0.9475091  0.0524909 ]
 [0.56180937 0.43819063]
 [0.31168059 0.68831941]
 [0.81032178 0.18967822]
 [0.74249378 0.25750622]
 [0.15346552 0.84653448]
 [0.91924619 0.08075381]
 [0.4240567  0.5759433 ]
 [0.73519914 0.26480086]
 [0.95500394 0.04499606]
 [0.70839169 0.29160831]
 [0.84329305 0.15670695]
 [0.88648519 0.11351481]]
                 Confusion matrix
                 Score positive    Score negative
Actual positive      3283               377
Actual negative       785               969

Accuracy  0.79
 
           Positive      Negative
Num case     3660          1754
Precision    0.81          0.72
Recall       0.90          0.55
F1           0.85          0.63


C:\Users\Michael Ikemann\Anaconda3\envs\lechler_ims\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Michael Ikemann\Anaconda3\envs\lechler_ims\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [123]:
from sklearn.ensemble import AdaBoostClassifier
ab_clf = AdaBoostClassifier(learning_rate=1.0, n_estimators=500)
ab_clf.fit(X_train, y_train)

probabilities = ab_clf.predict_proba(X_test)
scores = score_model(probabilities, 0.5)    
print_metrics(y_test, scores)

C:\Users\Michael Ikemann\Anaconda3\envs\lechler_ims\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                 Confusion matrix
                 Score positive    Score negative
Actual positive      3221               439
Actual negative       698              1056

Accuracy  0.79
 
           Positive      Negative
Num case     3660          1754
Precision    0.82          0.71
Recall       0.88          0.60
F1           0.85          0.65


In [124]:
import keras

In [125]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import L1L2

ohe = preprocessing.OneHotEncoder(categories='auto', sparse=False)
encoded = ohe.fit(y_train.reshape(-1,1))
yn_train = encoded.transform(y_train.reshape(-1,1))
yn_test = encoded.transform(y_test.reshape(-1,1))

print(y_test[:5])
print(yn_test[:5])

model = Sequential()
model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(2,activation='softmax', kernel_regularizer=L1L2(l1=0.0, l2=0.2),input_dim=X_train.shape[1]))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, yn_train, epochs=20, validation_data=(X_test, yn_test))

[[0]
 [0]
 [0]
 [1]
 [1]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]
Train on 10990 samples, validate on 5414 samples
Epoch 1/20
10990/10990 [==============================] - 2s 160us/step - loss: 0.6501 - acc: 0.7818 - val_loss: 0.4787 - val_acc: 0.7772
Epoch 2/20
10990/10990 [==============================] - 1s 122us/step - loss: 0.4652 - acc: 0.7848 - val_loss: 0.4677 - val_acc: 0.7830
Epoch 3/20
10990/10990 [==============================] - 1s 122us/step - loss: 0.4622 - acc: 0.7830 - val_loss: 0.4694 - val_acc: 0.7857
Epoch 4/20
10990/10990 [==============================] - 1s 122us/step - loss: 0.4630 - acc: 0.7862 - val_loss: 0.4746 - val_acc: 0.7789
Epoch 5/20
10990/10990 [==============================] - 1s 124us/step - loss: 0.4635 - acc: 0.7818 - val_loss: 0.4671 - val_acc: 0.7837
Epoch 6/20
10990/10990 [==============================] - 1s 124us/step - loss: 0.4599 - acc: 0.7857 - val_loss: 0.4678 - val_acc: 0.7819
Epoch 7/20
10990/10990 [===========================

In [126]:
probabilities = logistic_mod.predict_proba(Test_Features)
scores = score_model(probabilities, 0.5)
print(scores[:15])
print(scores.shape)

for score in scores:
    print(score)

[0 1 0 0 0 0 1 1 1 0 0 0 0 0 0]
(500,)
0
1
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
1
1
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
0
1
1
0
0
0
0
1
1
1
0
0
1
0
0
1
0
0
0
1
1
0
0
1
0
0
0
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
1
1
0
1
0
1
0
0
0
1
1
0
0
1
0
0
1
1
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
1
0
0
1
0
1
0
1
1
0
0
0
1
0
0
0
0
1
0
0
1
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
0
0
1
0
0
1
0
0
1
1
0
1
1
1
0
0
0
1
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
1
1
1
0
0
0
1
1
0
0
1
0
0
0
0
0
1
1
0
0
1
0
1
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
1
0
0
0
1
1
1
1
0
1
1
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
1
0
1
0
1
1
1
0
1
0
0
0
1
1
0
0
1
0
1
0
0
1
0
0
0
1
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
1
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
1
1
1
0
1
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
1
0
0
0
1
0
0
0
1
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
1
1
0
1
0
0
0
0
1
0
0
0
0
1
1
1
1